```glsl
out vec3 ray;
in vec3 position;
uniform float cameraFov;
uniform vec2 resolution;
uniform mat4 viewMatrix;

void main() {
  gl_Position = vec4(position.xy, 0, 1);
  float aspect = resolution.y / resolution.x;
  vec2 uv = vec2(position.x, position.y * aspect);
  float cameraDistance = (1.0 / tan(cameraFov / 2.0)) * aspect;
  ray = normalize(vec3(uv, -cameraDistance) * mat3(viewMatrix));
}
```

In [51]:
from sage.all import *
import math

from sage.combinat.output import box_exists


def normalize(v):
    magnitude = math.sqrt(sum(i ** 2 for i in range(len(v))))
    return vector((i/magnitude for i in range(len(v))))

def cross_product(a, b):
    # Promote 2D to 3D by adding z=0
    if len(a) == 2:
        a = [a[0], a[1], 0]
    if len(b) == 2:
        b = [b[0], b[1], 0]

    # Check dimension
    if len(a) != 3 or len(b) != 3:
        raise ValueError("Vectors must be 2D or 3D")

    # Compute 3D cross product
    return [
        a[1]*b[2] - a[2]*b[1],
        a[2]*b[0] - a[0]*b[2],
        a[0]*b[1] - a[1]*b[0]
    ]

def subtract(a, b):
    return [a[i] - b[i] for i in range(len(a))]

def dot(a, b):
    return sum(a[i]*b[i] for i in range(len(a)))

def look_at_matrix(eye, at, up):
    # Step 1: Compute z-axis
    zaxis = normalize(subtract(eye, at))

    # Step 2: Compute x-axis
    xaxis = normalize(cross_product(up, zaxis))

    # Step 3: Compute y-axis
    yaxis = cross_product(zaxis, xaxis)

    # Step 4: Create view matrix (right-handed)
    view = matrix([
        [xaxis[0], yaxis[0], zaxis[0], 0],
        [xaxis[1], yaxis[1], zaxis[1], 0],
        [xaxis[2], yaxis[2], zaxis[2], 0],
        [-dot(xaxis, eye), -dot(yaxis, eye), -dot(zaxis, eye), 1]
    ])

    return view

def mat4_to_mat3_sage(mat4):
    if mat4.nrows() != 4 or mat4.ncols() != 4:
        raise ValueError("Matrix must be 4x4")
    return mat4[0:3, 0:3]  # top-left 3x3 submatrix


In [52]:
from sage.all import *
cameraFov, resolution, viewMatrix, eye, at, up, position = var('cameraFov, resolution, viewMatrix, eye, at, up, position')
cameraFov = 2.0
resolution = vector([500,500])
up = vector([0.0,1.0,0.0])
at = vector([0.0,0.0,0.0])
eye = vector([0.0,0.5,1.0])
position = vector([0.0,0.0,0.0])
aspect = resolution[1]/resolution[0]
uv = vector([position[0], position[1] * aspect])
cameraDistance = (1.0 / tan(cameraFov / 2.0)) * aspect
viewMatrix = look_at_matrix(eye, at, up)
ray = normalize(vector([uv[0], uv[1], -cameraDistance]) * mat4_to_mat3_sage(viewMatrix))

In [74]:
@interact
def ray_plot(size=slider([0.1, 0.2, .. 1.0])):
    P = sphere(position, size=size, opacity = 0.5)
    C = sphere(eye, size = 0.1, legend_label="Camera", color = "red")
    L = line3d((ray, position), thickness=2.0, arrow_head=true, color="red", legend_label="ray")
    show(P + C + L, viewer='threejs').legend(True)

Interactive function <function ray_plot at 0x16de400e0> with 1 widget
  size: SelectionSlider(description='size', options=(0.100000000000000, 0.200000000000000, 0.300000000000000, 0.400000000000000, 0.500000000000000, 0.600000000000000, 0.700000000000000, 0.800000000000000, 0.900000000000000, 1.00000000000000), value=0.100000000000000)

In [54]:
from sage.all import *
@interact
def myplot(f=x**Integer(2)):
    show(plot(f,(x,-Integer(3),Integer(3))))

Interactive function <function myplot at 0x16e44a520> with 1 widget
  f: EvalText(value='x^2', description='f')